In [6]:
import pandas as pd
import numpy as np
from scipy.stats.distributions import gamma

from epimodel import read_csv
from modeling.nowcasting.nowcasting import exp_model

In [39]:
csse_ds = read_csv("../../data/CSSE.csv")

ICL_ITD = gamma(1/0.45**2,0,18.8*0.45**2)

DEATHS_DAYS = np.array([ICL_ITD.cdf(a+1)-ICL_ITD.cdf(a) for a in range(61)])

In [40]:
csse_ds.loc['BR','Confirmed'] = csse_ds.loc['BR','Confirmed'].shift(-7).values
csse_ds.loc['BR']

,Recovered,Confirmed,Deaths,Active
Date,,,,
2020-01-22 00:00:00+00:00,0.0,0.0,0.0,0.0
2020-01-23 00:00:00+00:00,0.0,0.0,0.0,0.0
2020-01-24 00:00:00+00:00,0.0,0.0,0.0,0.0
2020-01-25 00:00:00+00:00,0.0,0.0,0.0,0.0
2020-01-26 00:00:00+00:00,0.0,0.0,0.0,0.0
...,...,...,...,...
2020-04-03 00:00:00+00:00,127.0,NaN,359.0,8570.0
2020-04-04 00:00:00+00:00,127.0,NaN,445.0,9788.0
2020-04-05 00:00:00+00:00,127.0,NaN,486.0,10517.0


In [35]:
csse_ds.loc['BR','Confirmed'].shift(-7)

Date
2020-01-22 00:00:00+00:00    0.0
2020-01-23 00:00:00+00:00    0.0
2020-01-24 00:00:00+00:00    0.0
2020-01-25 00:00:00+00:00    0.0
2020-01-26 00:00:00+00:00    0.0
                            ... 
2020-04-03 00:00:00+00:00    NaN
2020-04-04 00:00:00+00:00    NaN
2020-04-05 00:00:00+00:00    NaN
2020-04-06 00:00:00+00:00    NaN
2020-04-07 00:00:00+00:00    NaN
Name: Confirmed, Length: 77, dtype: float64